# Progetto DSIM - Demo Live Immagini

---



Autori: Ginevra Mariani | Lorenzo Mora | Confalonieri Riccardo <br>
E-mail: g.mariani34@campus.unimib.it, l.mora4@campus.unimib.it, r.confalonieri5@campus.unimib.it

Lo scopo di questo notebook è implementare una live demo per dimostrare l'efficacia dei modelli implementati per il task di face recognition. Siccome la demo necessita della telecamera per acquisire le immagini è necessario eseguire il file localmente e non in Colab! Inoltre è importante controllare il path da cui si caricano i modelli trainati per evitare errori. <br>

**DISCLAIMER:** Nella finestra viene riportata anche la probabilità rispetto alla predizione del modello. Questo è valido soltanto sotto l'ipotesi di "mondo chiuso", ovvero se si considera che al modello non verranno mai forniti volti al di fuori di quelli del gruppo. Altrimenti questa probabilità non avrebbe alcun senso, e nel caso si volesse mantenerla sarebbe necessario riaddestrare i modelli per prevedere anche una classe "unknown".

## Import packages

In [ ]:
import cv2 as cv
# per gestire lo scatto della foto dal video
from cv2 import VideoCapture
# per caricare il modello CNN
from tensorflow import keras
# per pulire output console
from IPython.display import clear_output
# per operazioni con array
import numpy as np
import os # per gestire i path

## Demo
In questa demo viene visualizzata una finestra con le immagini catturate in real time attraverso la webcam principale del pc. A partire da queste immagini viene applicato il face detector di openCV, se viene trovato un volto il suo crop viene passato al modello per effettuare la predizione. <br>
Il risultato viene quindi visualizzato a video come testo

In [ ]:
def start_demo(face_detector, model, dict_names):
    """
    Visualizza una finestra con la riproduzione streaming catturata dalla camera principale
    del pc. Ad ogni frame viene quindi applicato il face detector e, se presente un volto,
    quest'ultimo viene passato al modello per riconoscere la persona.
    
    :param cv2.CascadeClassifier face_detector: Detector per il riconoscimento del volto
    :param keras.engine.functional.Functional model: Modello keras per la predizione
    :param str{} dict_names: Dizionario che associa nomi e numeri
    """
        
    # nel caso ci siano errori chiudo la finestra in automatico
    try:
        # initialize the camera
        cam = VideoCapture(0)

        while True:
            # acquisisco l'immagine dalla cam
            s, frame = cam.read()
            
            # se l'acquisizione non è andata a buon fine interrompo
            if not s:
                break
            else:
                # converto l'immagine in b/n
                img_bw = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
                
                # applico il face detector sull'immagine in b/n
                faces = face_detector.detectMultiScale(img_bw, 1.1, 8)

                for (x,y,w,h) in faces:
                    # visualizzo il rettangolo di detection
                    cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
                    
                    # estraggo il crop
                    crop = frame[y:y+h, x:x+w]
                    # converto il crop a colori RGB
                    crop = cv.cvtColor(crop, cv.COLOR_BGR2RGB)
                    # riscalo il crop in 224x224
                    crop = cv.resize(crop, (224, 224)) 
                    # aggiungo una dimensione
                    crop = np.expand_dims(crop, axis = 0)
                     
                    # applico il modello al crop
                    y_pred = model.predict(crop)
                    
                    # estraggo la predizione migliore
                    y_pred_max = np.argmax(y_pred)
                    
                    # ottengo la probabilità della previsione
                    y_pred = y_pred[0][y_pred_max]
                    
                    # se il modello predice con probabilità inferiore allora è incerto e quindi non visualizzo il testo
                    if y_pred > 0.6:
                        # visualizzo la previsione
                        cv.putText(frame, 
                               ("Person: " + str(dict_names[y_pred_max])) + ' - ' + str(np.round(y_pred,3)), 
                               ((x, y-5)), 
                                cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
                    
            
            # visualizzo la finestra
            cv.imshow('FaceRecognition', frame)

            # Termino il ciclo se viene premuto Q
            if cv.waitKey(20) & 0xFF == ord('q'):
                break
        
        # rilascio la cam e chiudo la finestra se il ciclo è finito
        cam.release()
        cv.destroyAllWindows()
        
    except: # rilascio la cam e chiudo la finestra
        cam.release()
        cv.destroyAllWindows()

In [ ]:
# carico il modello di face detection pre-addestrato
face_detector = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_frontalface_default.xml')
# carico il modello pre-trainato
#model = keras.models.load_model('./mobilenet.h5')
model = keras.models.load_model('./vggface.h5')

# definisco un dizionario con l'associazione Label-Nome
dict_name = {0: 'Ginevra', 1: 'Lorenzo', 2: 'Riccardo'}
start_demo(face_detector, model, dict_name)

## Considerazioni finali
L'accuracy della previsione dipende fortemente dal modello utilizzato ma anche dal detector, in alcuni casi infatti si è notato che opencv rileva come volti anche oggetti o la semplice porzione dell'occhio. Queste problematiche possono essere risolte usando dei detector più validi come `dlib` che non si è riusciti però a implementare in locela in quanto richiede una GPU. <br>
Inoltre un altro problema legato al detector è che ruotando il volto non riesce più ad individuare la faccia, in questo caso il modello non può quindi fornire una previsione. Facendo dei test croppando manualmente l'immagine si è però notato che il modello performa ugualmente anche con volti ruotati o inclinati. <br>
Infine si è notato che in alcuni casi il frame risulta rallentato, questo è dovuto a molteplici motivi:

* risorse hardware del pc
* tempo necessario per la detection
* tempo necessario per il recognition

Per quanto riguarda le ultime due casistiche il tempo si ridurrebbe drasticamente utilizzando una GPU!

In [2]:
# download notebook in .html extension
%%shell
jupyter nbconvert --to html '/content/drive/MyDrive/Confalonieri_Mariani_Mora_DSIM/Notebook/Processing-2D/3_DemoLive.ipynb'

[NbConvertApp] Converting notebook /content/drive/MyDrive/Confalonieri_Mariani_Mora_DSIM/Notebook/Processing-2D/3_DemoLive.ipynb to html
[NbConvertApp] Writing 292439 bytes to /content/drive/MyDrive/Confalonieri_Mariani_Mora_DSIM/Notebook/Processing-2D/3_DemoLive.html
